# GDELT Demo project showcase

This is the simple presentation of my findings and a bit about the skills involved in getting there. For gory details, see [README.md](https://github.com/reed9999/gdelt-demo/blob/master/README.md).

## Findings and visualizations

This project is still in early stages, but this is where I will present findings.

## Skills demonstrated

* **General quant research**: This will become evident in time. For now the best demonstration is my brainstorming about the research questions in [README.md](https://github.com/reed9999/gdelt-demo/blob/master/README.md).
* **Hive QL and SQL**: Several exploratory queries: 
  * For now the best examples are those having to do with the Goldstein scale column, see [queries/goldsteinscale](https://github.com/reed9999/gdelt-demo/tree/master/queries/goldsteinscale). Most of those should run as SQL against a local MySQL.
  
* **Spark** (via PySpark): See [queries/spark-sql](https://github.com/reed9999/gdelt-demo/tree/master/queries/spark-sql).
* **AWS setup** (S3, EMR cluster creation)
  * **via CLI**: See [automation/scripts](https://github.com/reed9999/gdelt-demo/tree/master/automation/scripts).
  * **via boto3**: See [automation](https://github.com/reed9999/gdelt-demo/tree/master/automation).